In [29]:
# =========================================================
# BLOCO 1 · IMPORTAÇÕES
# =========================================================
import sys

print(sys.executable)
import re
import os
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, PatternFill
from pathlib import Path

c:\Users\marcelo.petry\AppData\Local\Programs\Python\Python313\python.exe


In [30]:
# === PASTAS DO PROJETO ===

try:
    NB_DIR = Path(__file__).parent.resolve()
except NameError:
    NB_DIR = Path().resolve()  # Jupyter

REPO_ROOT = NB_DIR.parent
IN_DIR = (REPO_ROOT / "data").resolve()  # CSVs de entrada/intermediários
OUT_DIR = (REPO_ROOT / "outputs").resolve()  # XLSX finais

IN_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

print("IN_DIR =", IN_DIR)
print("OUT_DIR =", OUT_DIR)

IN_DIR = C:\Users\marcelo.petry\Documents\Vigiram\data
OUT_DIR = C:\Users\marcelo.petry\Documents\Vigiram\outputs


In [31]:
# =========================================================
# BLOCO 2 · CONSTANTES GERAIS
# =========================================================
ID_ESTAB = 260014  # Valor fixo p/ id_estabelecimento
ID_LAB = 260008  # Valor fixo p/ id_laboratorio
NU_AMOSTRA = "U"  # Sempre "U" (amostra única)

PT_MES = [
    "",
    "jan",
    "fev",
    "mar",
    "abr",
    "mai",
    "jun",
    "jul",
    "ago",
    "set",
    "out",
    "nov",
    "dez",
]

In [32]:
# =========================================================
# BLOCO 3 · FUNÇÃO LEITURA CSV
# =========================================================
def read_csv_robusto(path_csv: str, sep: str = ",") -> pd.DataFrame:
    for enc in ("utf-8-sig", "utf-8", "latin-1", "cp1252"):
        try:
            return pd.read_csv(
                path_csv, sep=sep, dtype=str, encoding=enc, engine="python"
            )
        except UnicodeDecodeError:
            continue

    # Último recurso: substitui caracteres inválidos
    return pd.read_csv(
        path_csv,
        sep=sep,
        dtype=str,
        encoding="utf-8",
        encoding_errors="replace",
        engine="python",
    )

In [33]:
# =========================================================
# BLOCO 4 · DICIONÁRIOS DE MATERIAL, MICROORGANISMOS, ANTIBIÓTICOS
# =========================================================

MAP_MATERIAL = {
    "ABSCESSO": 1,
    "ASPIRADO TRAQUEAL": 7,
    "DIVERSOS": 0,
    "ESCARRO POR EXPECTORAÇÃO": 14,
    "FEZES": 26,
    "LAVADO BRONCOALVEOLAR": 56,
    "LIQUOR": 74,
    "NAO COLETAVEL": 0,
    "OUTROS (ESPECIFICAR)": 0,
    "SANGUE": 94,
    "SECREÇÃO": 98,
    "SECREÇÃO DE FERIDA OPERATÓRIA": 25,
    "SORO": 121,
    "SWAB ANAL": 123,
    "SWAB AXILAR E INGUINAL": 122,
    "SWAB ESTÉRIL": 122,
    "SWAB NASAL": 132,
    "SWAB RETAL": 136,
    "URINA": 142,
    "URINA CATETERIZADA": 142,
    "URINA POR JATO MEDIO": 142,
}

MAP_MICRO = {
    "ABSIDIA SP.": 10002,
    "ACINETOBACTER BAUMANNII": 31,
    "ACINETOBACTER LWOFFII": 37,
    "ACINETOBACTER SP": 42,
    "ACHROMOBACTER SP.": 16,
    "ACHROMOBACTER XYLOSOXIDANS": 17,
    "AEROCOCCUS": 92,
    "AEROMONAS HYDROPHILA/CAVIAE": 100,
    "AEROMONAS SP.": 106,
    "ALCALIGENES FAECALIS": 121,
    "BACILO GRAM NEGATIVO FERMENTADOR": "N/A",
    "BACILO GRAM POSITIVO": 2314,
    "BACILLUS SP.": 252,
    "BACTEROIDES FRAGILIS": 264,
    "BORDETELLA BRONCHISEPTICA": 312,
    "BURKHOLDERIA CEPACIA COMPLEXO": 388,
    "BURKHOLDERIA PSEUDOMALLEI": 2326,
    "CAMPYLOBACTER JEJUNI SUBSP. JEJUNI": 427,
    "CANDIDA ALBICANS": 10065,
    "CANDIDA DUBLINIENSIS": 10248,
    "CANDIDA GLABRATA(T. GLABRATA)": 10254,
    "CANDIDA GUILLIERMONDII": 10067,
    "CANDIDA KRUSEI": 10070,
    "CANDIDA PARAPSILOSIS": 10074,
    "CANDIDA TROPICALIS": 10078,
    "CHRYSEOBACTERIUM INDOLOGENES": 491,
    "CHRYSEOBACTERIUM SP.": 492,
    "CITROBACTER FREUNDII": 500,
    "CITROBACTER KOSERI": 501,
    "CITROBACTER SPECIES": 504,
    "COCOS GRAM POSITIVOS EM CADEIA": "N/A",
    "ELIZABETHKINGIA MENINGOSEPTICA": 719,
    "ENTEROBACTER AEROGENES": 724,
    "ENTEROBACTER ASBURIAE": 729,
    "ENTEROBACTER CLOACAE": 731,
    "ENTEROBACTER SP.": 740,
    "ENTEROCOCCUS AVIUM - (GRUPO D)": 741,
    "ENTEROCOCCUS DURANS - (GRUPO D)": 745,
    "ENTEROCOCCUS FAECALIS - (GRUPO D)": 746,
    "ENTEROCOCCUS FAECIUM - (GRUPO D)": 747,
    "ENTEROCOCCUS GALLINARUM": 748,
    "ENTEROCOCCUS RAFFINOSUS": 751,
    "ENTEROCOCCUS SP.": 753,
    "ERYSIPELOTHRIX RHUSIOPATHIAE": 763,
    "ESCHERICHIA COLI": 767,
    "ESCHERICHIA COLI (CEPA 2)": 767,
    "ESCHERICHIA COLI (PRODUTORA DE ESBL)": 2290,
    "HAEMOPHILUS INFLUENZAE": 910,
    "HAEMOPHILUS PARAINFLUENZAE": 920,
    "KLEBISIELLA SP": 1014,
    "KLEBSIELLA OXYTOCA": 1007,
    "KLEBSIELLA OXYTOCA (PRODUTORA DE ESBL)": 2291,
    "KLEBSIELLA OZAENAE": 1008,
    "KLEBSIELLA PNEUMONIAE": 1010,
    "KLEBSIELLA PNEUMONIAE (PRODUTORA DE ESBL)": 2292,
    "KLEBSIELLA SP.": 1014,
    "LEUCONOSTOC SP.": 1121,
    "LISTERIA SP.": 1133,
    "MICROCOCCUS SP.": 1217,
    "MORAXELLA (BRANHAMELLA) CATARRHALIS": 2424,
    "MORAXELLA SP.": 1243,
    "MORGANELLA MORGANII": 1244,
    "PANTEA AGGLOMERANS": 1541,
    "PANTEA SP.": 1543,
    "PROTEUS MIRABILIS": 1666,
    "PROTEUS PENNERI": 1667,
    "PROTEUS SP.": 1668,
    "PROTEUS VULGARIS": 1669,
    "PROVIDENCIA ALCALIFACIENS": 1674,
    "PROVIDENCIA RETTGERI": 1675,
    "PROVIDENCIA STUARTII": 1678,
    "PSEUDOMONAS AERUGINOSA": 1684,
    "PSEUDOMONAS FLUORESCENS": 1689,
    "PSEUDOMONAS MENDOCINA": 1692,
    "PSEUDOMONAS OLEOVORANS": 1693,
    "PSEUDOMONAS PUTIDA": 1697,
    "PSEUDOMONAS SP.": 1699,
    "PSEUDOMONAS STUTZERI": 1700,
    "SALMONELLA SP.": 1856,
    "SERRATIA FONTICOLA": 1881,
    "SERRATIA MARCESCENS": 1884,
    "SERRATIA ODORIFERA": 1885,
    "SERRATIA RUBIDAEA": 1889,
    "SHIGELLA FLEXNERI": 1896,
    "SHIGELLA SONNEI": 1898,
    "SHIGELLA SP.": 1899,
    "SPHINGOMONAS PAUCIMOBILIS": 1929,
    "STAPHYL. SP. COAGULASE NEG. (CEPA 2)": 1961,
    "STAPHYLOCOCCUS AUREUS": 1952,
    "STAPHYLOCOCCUS AUREUS (MRSA)": 2289,
    "STAPHYLOCOCCUS CAPITIS": 1955,
    "STAPHYLOCOCCUS COHNII": 4064,
    "STAPHYLOCOCCUS EPIDERMIDIS": 1965,
    "STAPHYLOCOCCUS HAEMOLYTICUS": 1968,
    "STAPHYLOCOCCUS HOMINIS": 1969,
    "STAPHYLOCOCCUS LENTUS": 1976,
    "STAPHYLOCOCCUS SAPROPHYTICUS": 1981,
    "STAPHYLOCOCCUS SP": 1987,
    "STAPHYLOCOCCUS SP. COAGULASE NEGATIVA": 1961,
    "STAPHYLOCOCCUS SP. COAGULASE POSITIVA": "N/A",
    "STAPHYLOCOCCUS WARNERI": 1989,
    "STENOTROPHOMONAS MALTOPHILIA": 1995,
    "STREPT. BETA HEMOLITICO GRUPO B": 2012,
    "STREPT. INTERMEDIUS(VIRIDANS STREPT.)": 2039,
    "STREPT. MITIS (VIRIDANS STREPT.)": 2041,
    "STREPT. SALIVARIUS (VIRIDANS STREPT.)": 2053,
    "STREPTOCOCCUS AGALACTIAE - ( GRUPO B )": 2005,
    "STREPTOCOCCUS PNEUMONIAE": 2049,
    "STREPTOCOCCUS PYOGENES-(GROUP A)": 2052,
    "STREPTOCOCCUS SP (GRUPO VIRIDANS)": 2068,
    "TRICHOSPORON ASAHII": 10268,
    "VIBRIO CHOLERAE": 2217,
    "VIBRIO CHOLERAE NÃO O1": 2218,
    "VIBRIO PARAHAEMOLYTICUS": 2233,
    "YERSINIA ENTEROCOLITICA": 2271,
    "STREPTOCOCCUS SP. ALFA HEMOLITICO": 4091,
    "CHROMOBACTERIUM VIOLACEUM": 486,
    "STREPTOCOCCUS SP. BETA HEMOLÍTICO": 2010,
    "CANDIDA SP.": 10076,
    "BACILO GRAM NEGATIVO NÃO FERMENTADOR": 2313,
    "STREPTOCOCCUS SANGUINIS": 2056,
    "ACINETOBACTER JUNII": 36,
    "VERSÃO NÃO PARAMETRIZADA": "",
    "ENTEROCOCCUS GRUPO D": 753,
    "S. SAPROPHYTICUS/S. HOMINIS": 1981,
    "ACINETOBACTER JOHNSONII": 35,
    "RHODOTORULA SP.": 10194,
    "KODAMAEA OHMERI": 10307,
    "CANDIDA AURIS": 10328,
    "KLUYVERA ASCORBATA": 1016,
    "PANDORAEA SP.": 1539,
    "STREPTOCOCCUS GALLOLYTICUS": 2031,
    "STREPTOCOCCUS SP.": 2059,
    "RALSTONIA MANNITOLILYTICA": 1720,
    "PANTOEA SP.": 1543,
    "MICROCOCCUS LUTEUS": 1214,
}

MAP_AB = {
    "OXACILINA": 131,
    "GENTAMICINA": 96,
    "LEVOFLOXACINA": 105,
    "ERITROMICINA": 77,
    "CLINDAMICINA": 63,
    "LINEZOLIDE": 108,
    "DAPTOMICINA": 69,
    "TEICOPLANINA": 158,
    "TEICOPLAMINA": 158,
    "VANCOMICINA": 176,
    "VANCOMYCINA": 176,
    "TIGECICLINA": 166,
    "RIFAMPICINA": 146,
    "TRIMETROPIN SULFAMETOXAZOL": 151,
    "PIPERACILINA TAZOBACTAM": 135,
    "AMPICILINA": 9,
    "AMPICILINA-SULBACTAM": 10,
    "AMPICILINA - SULBACTAN": 10,
    "CEFTAZIDIMA": 50,
    "CEFTRIAXONA": 56,
    "ERTAPENEM": 78,
    "MEROPENEM": 113,
    "AMICACINA": 5,
    "CIPROFLOXACINA": 60,
    "CIPROFLOXACINO": 60,
    "AZTREONAM": 13,
    "COLISTINA": 67,
    "FLUCONAZOL": 182,
    "VORICONAZOL": 186,
    "CASPOFUNGINA": 179,
    "MICAFUNGINA": 180,
    "ANFOTERICINA B": 177,
    "AMOXACILINA-ÁCIDO CLAVULÂNICO": 8,
    "AMOXACILINA-CLAVULANATO": 8,
    "CLORANFENICOL": 65,
    "FOSFOMICINA": 91,
    "TETRACICLINA": 163,
    "TOBRAMICINA": 170,
    "CEFUROXINA": 57,
    "CEFOXITINA": 42,
    "CEFTAROLINE": 48,
    "CEFTAZIDIMA / AVIBACTAM": 51,
    "CEFTAZOLINA": 27,
    "CEFTOLOZANE / TAZOBACTAM": 55,
    "CEFTOZOLIN": 27,
    "CEFOTAXIMA": 39,
    "CEFEPIME": 30,
    "PENICILINA G": 133,
    "GENTAMICINA SYNERGY": 97,
    "NORFLOXACIN": 127,
    "NITROFURANTOINA": 126,
    "CEFTAZIDIME": 50,
    "STREPTOMICINA SYNERGY": 82,
    "CEFALEXINA": 23,
    "IMIPENEM": 101,
    "CEFUROXIMA AXETIL": 57,
    "CHLORANFENICOL": 65,
    "CIPROFLOXACINA (MENINGITE)": 29,
}

In [34]:
# =========================================================
# BLOCO 5 · MAP_UNIDADE (unidade_solicitante => co_unidade_origem)
# =========================================================
MAP_UNIDADE = {
    "10º NORTE": 8,
    "10º SUL": 8,
    "11º NORTE": 8,
    "11º SUL": 8,
    "5º NORTE": 8,
    "6º SUL": 8,
    "7 º SUL": 8,
    "7º NORTE": 8,
    "8º NORTE": 8,
    "8º SUL": 8,
    "9º  SUL": 8,
    "9º NORTE": 8,
    "ACUPUNTURA (AMBULATÓRIO)": 7,
    "ALERGIA E IMUNOLOGIA (AMBULATÓRIO)": 7,
    "BLOCO OBSTETRICO": 8,
    "CARDIOLOGIA (AMBULATÓRIO)": 7,
    "CCIH (AMBULATÓRIO)": 7,
    "CIRURGIA DE CABECA E PESCOCO": 8,
    "CIRURGIA GERAL (AMBULATÓRIO)": 7,
    "CIRURGIA PLÁSTICA (AMBULATÓRIO)": 7,
    "CIRURGIA TORÁCICA (AMBULATÓRIO)": 7,
    "CIRURGIA VASCULAR (AMBULATÓRIO)": 7,
    "CLÍNICA MÉDICA (AMBULATÓRIO)": 7,
    "DERMATOLOGIA (AMBULATÓRIO)": 7,
    "ENFERMAGEM": 8,
    "GASTROENTEROLOGIA (AMBULATÓRIO)": 7,
    "GERIATRIA (AMBULATORIO)": 7,
    "GINECOLOGIA (AMBULATÓRIO)": 7,
    "HEMATOLOGIA (AMBULATÓRIO)": 7,
    "HEMODIALISE": 0,
    "HOSPITAL - DIA": 0,
    "INFECTOLOGIA (AMBULATÓRIO)": 7,
    "NEFROLOGIA (AMBULATÓRIO)": 7,
    "NEUROLOGIA (AMBULATÓRIO)": 7,
    "NÚCLEO DO SERVIDOR (AMBULATÓRIO)": 7,
    "OBSTETRÍCIA (AMBULATÓRIO)": 7,
    "OFTALMO GERAL": 0,
    "OFTALMOLOGIA (AMBULATÓRIO)": 7,
    "ONCOLOGIA (AMBULATÓRIO)": 7,
    "ORTOPEDIA (AMBULATÓRIO)": 7,
    "OTORRINOLARINGOLOGIA (AMBULATÓRIO)": 7,
    "PEDIATRIA (AMBULATÓRIO)": 7,
    "PNEUMOLOGIA (AMBULATÓRIO)": 7,
    "PSIQUIATRIA (AMBULATÓRIO)": 7,
    "PUERICULTURA (AMBULATÓRIO)": 7,
    "REUMATOLOGIA (AMBULATÓRIO)": 7,
    "SAUDE OCUPACIONAL E SEGURANCA DO TRABALHO": 0,
    "TRANSPLANTE (AMBULATÓRIO)": 7,
    "UCI CANGURU": 5,
    "UCI NEONATAL": 5,
    "UNIDADE AMBULATORIAL": 7,
    "URGENCIA E EMERGENCIA": 6,
    "UROLOGIA (AMBULATÓRIO)": 7,
    "UTI ADULTO": 1,
    "UTI CLINICA": 12,
    "UTI NEONATAL": 3,
}

In [35]:
# =========================================================# BLOCO 3½ · LISTAS-CHAVE PARA FILTROS CLÍNICOS (globais)# =========================================================# Estes arrays são usados em gerar_excel_brglass e gerar_consolidado_diario# para identificar amostras Carbapenemase, VRE e MRSA.MIC_CARBA_VALIDOS = [    "1684", "1699", "1700", "1697", "1689", "1693", "31", "37", "42",    "35", "36", "1667", "1010", "731", "1666", "724", "1889", "1675",    "1007", "1678", "500", "1884", "740", "501", "1881", "504", "1014",    "1668", "1674", "1669", "1543", "729", "1008", "767", "1885", "1244",    "2290", "2291", "2292", "1541", "1856", "1883", "1896", "1898",    "1899", "2271",]MIC_VRE_VALIDOS = ["746", "747"]   # Enterococcus faecalis / faeciumMIC_MRSA_VALIDO = "1952"           # Staphylococcus aureus

In [36]:
# =========================================================
# BLOCO 6 · FUNÇÕES DE MAPEAMENTO
# =========================================================


def _norm(txt):
    """Normaliza texto: strip + upper. Se for NaN, retorna string vazia."""
    return str(txt).strip().upper() if pd.notna(txt) else ""


def map_tp_atendimento(r):
    txt = _norm(r["origem"])
    if txt.startswith("AMBUL"):
        return "A"
    return "H"


def map_co_proc_amostra(r):
    return 1 if map_tp_atendimento(r) == "A" else 2


def map_desfecho(r):
    t = _norm(r["Tipo Alta"])
    if t in (
        "ALTA MÉDICA",
        "ALTA DA MÃE - PUÉRPERA E PERMANÊNCIA DO RECÉM-NASCIDO",
        "ALTA DA MÃE - PUÉRPERA E DO RECÉM-NASCIDO",
        "ALTA",
    ):
        return 1
    if "TRANSFERÊNCIA" in t:
        return 2
    if "OBITO" in t or "ÓBITO" in t:
        return 3
    if t in ("", "EVASÃO"):
        return 5
    return 5


def map_id_material(r):
    return MAP_MATERIAL.get(_norm(r["descmaterial_analise"]), 0)


def map_micro_id(r):
    return MAP_MICRO.get(_norm(r["microorganismo"]), _norm(r["microorganismo"]))


def map_ab_id(r):
    return MAP_AB.get(_norm(r["Antibiótico"]), _norm(r["Antibiótico"]))


def map_metodo_tsa(r):
    txt = _norm(r["Mic"])
    vazios = {"", "NAN", "NA", "N/A", "*", "-", "ND", "SYN-S"}
    if txt in vazios:
        return 1  # Difusão em disco
    return 8  # Automação


def map_rs(r):
    if "RSI" in r:
        if _norm(r["RSI"]) == "SAE":
            return "I"
    txt = _norm(r["RSI"])
    if txt in ("R", "S", "I"):
        return txt
    if "SAE" in txt:
        return "S"
    return txt


def map_vl(r):
    v = str(r["Mic"])
    if v.lower() == "nan":
        return ""
    return v


def map_co_unidade(r):
    unid = _norm(r["unidade_solicitante"])
    return MAP_UNIDADE.get(unid, 0)


# Dados do paciente
def map_co_paciente(r):
    return r["codigo_paciente"]


def map_nr_cns(r):
    return r["nro_cartao_saude"]


def map_no_paciente(r):
    return r["paciente"]


def map_no_mae(r):
    return r["nome_mae"]


def map_dt_nasc(r):
    return r["dt_nascimento"]


def map_genero(r):
    return r["sexo"]


def map_mun(r):
    return r["id_municipio"]


def map_uf(r):
    return r["uf_sigla"]


def map_co_amostra(r):
    """
    co_amostra = solicitacao + Item Solicitado (3 dígitos).
    Ex: solicitacao=140702, item=1 => "140702001" (9 dígitos).
    """
    solicit = str(r["solicitacao"]).strip()
    item = str(r["Item Solicitado"]).strip() if pd.notna(r["Item Solicitado"]) else "0"
    try:
        item_num = int(item)
    except:
        item_num = 0
    item_str = f"{item_num:03d}"
    return solicit + item_str

In [37]:
# =========================================================
# BLOCO 7 · DATAFRAMES FIXOS / VAZIOS
# =========================================================
def df_estabelecimento(mes: int, ano: int) -> pd.DataFrame:
    """
    Registro 00: Estabelecimento
    """
    return pd.DataFrame(
        {
            "nr_registro": ["00"],
            "id_estabelecimento": [ID_ESTAB],
            "co_matriz": ["HU"],
            "nr_mes_referencia": [mes],
            "nr_ano_referencia": [ano],
            "tp_laboratorio": [1],  # 1 = interno
            "id_laboratorio": [ID_LAB],
            "tp_envio": [0],  # 0 = teste
            "tp_arquivo": [1],  # 1 = amostras
            "vs_arquivo": ["2.14"],
        }
    )


def df_vazio(colunas) -> pd.DataFrame:
    return pd.DataFrame(columns=list(colunas))

In [38]:
# =========================================================
# BLOCO 8 · PROCESSAR CSV => R10..R50
# =========================================================
def processar_csv(csv_path: str):
    df = read_csv_robusto(csv_path, sep=",")

    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)

    # R10: Paciente
    pac = (
        df.groupby("codigo_paciente", as_index=False)
        .first()
        .assign(
            nr_registro="10",
            id_estabelecimento=ID_ESTAB,
            co_paciente=lambda x: x.apply(map_co_paciente, axis=1),
            nr_cns=lambda x: x.apply(map_nr_cns, axis=1),
            no_paciente=lambda x: x.apply(map_no_paciente, axis=1),
            no_mae=lambda x: x.apply(map_no_mae, axis=1),
            dt_nascimento=lambda x: x.apply(map_dt_nasc, axis=1),
            co_genero=lambda x: x.apply(map_genero, axis=1),
            id_municipio=lambda x: x.apply(map_mun, axis=1),
            ds_siglaUF=lambda x: x.apply(map_uf, axis=1),
        )
    )
    R10_cols = [
        "nr_registro",
        "id_estabelecimento",
        "co_paciente",
        "nr_cns",
        "no_paciente",
        "no_mae",
        "dt_nascimento",
        "co_genero",
        "id_municipio",
        "ds_siglaUF",
    ]
    df_r10 = pac[R10_cols]

    # R20: Atendimento
    at = (
        df.groupby("solicitacao", as_index=False)
        .first()
        .assign(
            nr_registro="20",
            id_estabelecimento=ID_ESTAB,
            co_atendimento=lambda x: x["solicitacao"],
            co_paciente=lambda x: x["codigo_paciente"],
            dt_atendimento=lambda x: x["inicio_atendimento"],
            tp_atendimento=lambda x: x.apply(map_tp_atendimento, axis=1),
            dt_internacao=lambda x: x["inicio_atendimento"],
            co_procedencia_amostra=lambda x: x.apply(map_co_proc_amostra, axis=1),
            co_unidade_origem=lambda x: x.apply(map_co_unidade, axis=1),
            co_desfecho=lambda x: x.apply(map_desfecho, axis=1),
        )
    )
    R20_cols = [
        "nr_registro",
        "id_estabelecimento",
        "co_atendimento",
        "co_paciente",
        "dt_atendimento",
        "tp_atendimento",
        "dt_internacao",
        "co_procedencia_amostra",
        "co_unidade_origem",
        "co_desfecho",
    ]
    df_r20 = at[R20_cols]

    # R30: Amostra
    am = (
        df.groupby(["solicitacao", "Item Solicitado"], as_index=False)
        .first()
        .assign(
            nr_registro="30",
            id_estabelecimento=ID_ESTAB,
            co_atendimento=lambda x: x["solicitacao"],
            co_paciente=lambda x: x["codigo_paciente"],
            co_amostra=lambda x: x.apply(map_co_amostra, axis=1),
            id_material=lambda x: x.apply(map_id_material, axis=1),
            nu_amostra=NU_AMOSTRA,
            dt_cadastro=lambda x: x["data_area_executora"],
            dt_coleta=lambda x: x["inicio_atendimento"],
            dt_liberacao=lambda x: x["dthr_liberada"],
            ds_motivo=lambda x: x["desc_regiao_anatomica"],
        )
    )
    R30_cols = [
        "nr_registro",
        "id_estabelecimento",
        "co_atendimento",
        "co_paciente",
        "co_amostra",
        "id_material",
        "nu_amostra",
        "dt_cadastro",
        "dt_coleta",
        "dt_liberacao",
        "ds_motivo",
    ]
    df_r30 = am[R30_cols]

    # R40: Microrganismo
    mic = (
        df.groupby(["solicitacao", "Item Solicitado", "microorganismo"], as_index=False)
        .first()
        .assign(
            nr_registro="40",
            id_estabelecimento=ID_ESTAB,
            co_atendimento=lambda x: x["solicitacao"],
            co_amostra=lambda x: x.apply(map_co_amostra, axis=1),
            id_microrganismo=lambda x: x.apply(map_micro_id, axis=1),
            co_perfil=1,
        )
    )
    R40_cols = [
        "nr_registro",
        "id_estabelecimento",
        "co_atendimento",
        "co_amostra",
        "id_microrganismo",
        "co_perfil",
    ]
    df_r40 = mic[R40_cols]

    # R50: TSA (corrigido para eliminar duplicidades globais)
    tsa = df.assign(
        nr_registro="50",
        id_estabelecimento=ID_ESTAB,
        co_atendimento=lambda x: x["solicitacao"],
        co_amostra=lambda x: x.apply(map_co_amostra, axis=1),
        id_microrganismo=lambda x: x.apply(map_micro_id, axis=1),
        id_antimicrobiano=lambda x: x.apply(map_ab_id, axis=1),
        id_criterio_tsa=1,
        id_metodo_tsa=lambda x: x.apply(map_metodo_tsa, axis=1),
        rs_tsa=lambda x: x.apply(map_rs, axis=1),
        vl_tsa=lambda x: x.apply(map_vl, axis=1),
    )

    R50_cols = [
        "nr_registro",
        "id_estabelecimento",
        "co_atendimento",
        "co_amostra",
        "id_microrganismo",
        "id_antimicrobiano",
        "id_criterio_tsa",
        "id_metodo_tsa",
        "rs_tsa",
        "vl_tsa",
    ]
    df_r50 = tsa[R50_cols].copy()

    # Remove registros "NÃO TESTADO"
    df_r50 = df_r50[~df_r50["rs_tsa"].str.strip().str.upper().eq("NÃO TESTADO")]

    # **Elimina Duplicidades Reais (todas colunas)**
    df_r50 = df_r50.drop_duplicates()

    return df_r10, df_r20, df_r30, df_r40, df_r50

In [39]:
# =========================================================
# BLOCO 9 · FORMATAÇÃO DA PLANILHA
# =========================================================
def formatar_planilha(path_xlsx: str) -> None:
    """
    Formata:
    - Cabeçalho: fundo cinza (D9D9D9), Arial 10, centralizado
    - Corpo: Arial 10, centralizado
    """
    wb = load_workbook(path_xlsx)
    fill = PatternFill("solid", fgColor="D9D9D9")
    fonte = Font(name="Arial", size=10)
    alin = Alignment(horizontal="center", vertical="center")

    for ws in wb.worksheets:
        # Cabeçalho
        for c in ws[1]:
            c.fill = fill
            c.font = fonte
            c.alignment = alin
        # Dados
        for row in ws.iter_rows(min_row=2):
            for c in row:
                c.font = fonte
                c.alignment = alin

    wb.save(path_xlsx)

In [40]:
# =========================================================
# BLOCO 10 · _extrair_mes_ano2  +  gerar_consolidado_diario  +  gerar_excel_brglass
# =========================================================


def _extrair_mes_ano2(nm_arq: str):
    """
    Extrai (mês, ano) de nomes como 'vigiram-jan23.csv' → (1, 2023).
    Retorna (0, 0) se não reconhecer o padrão.
    """
    import re, os

    base = os.path.splitext(os.path.basename(nm_arq))[0]
    m = re.search(r"vigiram-([a-z]{3})(\d{2})", base.lower())
    if not m:
        return 0, 0
    mes_txt, ano2d = m.groups()
    mes_num = PT_MES.index(mes_txt)
    return mes_num, 2000 + int(ano2d)


# ---------------------------------------------------------------------------
# GARANTIA: listas globais existem mesmo que a célula de dicionários não rode
# ---------------------------------------------------------------------------
MIC_CARBA_VALIDOS = globals().get(
    "MIC_CARBA_VALIDOS",
    [
        "1684",
        "1699",
        "1700",
        "1697",
        "1689",
        "1693",
        "31",
        "37",
        "42",
        "35",
        "36",
        "1667",
        "1010",
        "731",
        "1666",
        "724",
        "1889",
        "1675",
        "1007",
        "1678",
        "500",
        "1884",
        "740",
        "501",
        "1881",
        "504",
        "1014",
        "1668",
        "1674",
        "1669",
        "1543",
        "729",
        "1008",
        "767",
        "1885",
        "1244",
        "2290",
        "2291",
        "2292",
        "1541",
        "1856",
        "1883",
        "1896",
        "1898",
        "1899",
        "2271",
    ],
)
MIC_VRE_VALIDOS = globals().get("MIC_VRE_VALIDOS", ["746", "747"])
MIC_MRSA_VALIDO = globals().get("MIC_MRSA_VALIDO", "1952")
# ---------------------------------------------------------------------------


# ---------------------------------------------------------------------------
# Função: gerar_consolidado_diario
# ---------------------------------------------------------------------------
def gerar_consolidado_diario(
    df_r30: pd.DataFrame,
    df_R60: pd.DataFrame,
    df_R70: pd.DataFrame,
    df_R80: pd.DataFrame,
    df_r50: pd.DataFrame,
    *,
    mes_alvo: int,
    ano_alvo: int,
) -> pd.DataFrame:
    """
    Cria a aba 'Consolidado diário' restrita ao mês/ano solicitados.
    """

    # 1. Lê CSVs de amostras negativas/positivas (autodetecta vírgula ou ';') e agrega por dia (mês/ano alvo)
    negativos_csv: dict[str, int] = {}
    positivos_csv: dict[str, int] = {}

    def _carregar_agregado_por_dia(caminho: str) -> dict[str, int]:
        df = pd.read_csv(
            caminho,
            sep=None,
            engine="python",
            dtype=str,
            encoding="utf-8",
        )
        df.columns = df.columns.str.strip().str.lower()
        # Detecta coluna de data
        possiveis = ["dthr_entrada", "dia", "data", "data_entrada"]
        data_col = next((c for c in possiveis if c in df.columns), None)
        if not data_col:
            raise KeyError(f"CSV '{caminho}' não tem coluna de data ({possiveis})")
        # Converte e filtra mês/ano
        df[data_col] = pd.to_datetime(df[data_col], dayfirst=True, errors="coerce")
        df = df.dropna(subset=[data_col])
        df = df[
            (df[data_col].dt.month == mes_alvo) & (df[data_col].dt.year == ano_alvo)
        ]
        # Normaliza coluna contagem
        if "contagem" not in df.columns:
            raise KeyError(f"CSV '{caminho}' não tem coluna 'Contagem'")
        df["contagem"] = (
            pd.to_numeric(df["contagem"], errors="coerce").fillna(0).astype(int)
        )
        df["dt_fmt"] = df[data_col].dt.strftime("%d/%m/%Y")
        return df.groupby("dt_fmt")["contagem"].sum().astype(int).to_dict()

    try:
        negativos_csv = _carregar_agregado_por_dia(
            IN_DIR / "Contagem pacientes amostras negativas.csv"
        )

    except Exception as e:
        print("❌ Erro ao ler 'Contagem pacientes amostras negativas.csv':", e)

    try:
        positivos_csv = _carregar_agregado_por_dia(
            IN_DIR / "Contagem pacientes amostra positiva.csv"
        )

    except Exception as e:
        print("❌ Erro ao ler 'Contagem pacientes amostra positiva.csv':", e)
    # 2. Filtra R30 para mês/ano alvo
    df_r30["dt_cadastro"] = pd.to_datetime(
        df_r30["dt_cadastro"], dayfirst=True, errors="coerce"
    )
    df_r30 = df_r30[
        (df_r30["dt_cadastro"].dt.month == mes_alvo)
        & (df_r30["dt_cadastro"].dt.year == ano_alvo)
    ]
    datas_r30 = df_r30["dt_cadastro"].dt.strftime("%d/%m/%Y").dropna().unique()
    todas_datas = sorted(
        set(datas_r30) | set(negativos_csv) | set(positivos_csv),
        key=lambda d: datetime.strptime(d, "%d/%m/%Y"),
    )

    # 3. Monta registros dia-a-dia
    registros = []
    for data in todas_datas:
        amostras = df_r30.loc[
            df_r30["dt_cadastro"].dt.strftime("%d/%m/%Y") == data, "co_amostra"
        ].unique()
        qt_pacientes = df_r30.loc[
            df_r30["dt_cadastro"].dt.strftime("%d/%m/%Y") == data, "co_paciente"
        ].nunique()

        # positivos
        qt_vre_pos = df_R70[df_R70["co_amostra"].isin(amostras)]["co_amostra"].nunique()
        qt_mrsa_pos = df_R80[df_R80["co_amostra"].isin(amostras)][
            "co_amostra"
        ].nunique()
        qt_carba_pos = df_R60[df_R60["co_amostra"].isin(amostras)][
            "co_amostra"
        ].nunique()

        # negativos helper
        def _neg(df, mic_list, ab):
            return df[
                (df["co_amostra"].isin(amostras))
                & (df["id_microrganismo"].astype(str).isin(mic_list))
                & (df["id_antimicrobiano"].astype(str).eq(ab))
                & (df["rs_tsa"].str.upper().eq("S"))
            ]["co_amostra"].nunique()

        qt_vre_neg = _neg(df_r50, MIC_VRE_VALIDOS, "176")
        qt_mrsa_neg = _neg(df_r50, [MIC_MRSA_VALIDO], "131")
        qt_carba_neg = _neg(df_r50, MIC_CARBA_VALIDOS, "113")

        registros.append(
            {
                "nr_registro": "99",
                "id_estabelecimento": ID_ESTAB,
                "dt_consolidado": data,
                "qt_pacientes": qt_pacientes,
                "qt_cultura_negativa": negativos_csv.get(data, 0),
                "qt_cultura_positiva": positivos_csv.get(data, 0),
                "qt_carbapenemase_pesquisado": qt_carba_pos + qt_carba_neg,
                "qt_carbapenemase_negativo": qt_carba_neg,
                "qt_carbapenemase_positivo": qt_carba_pos,
                "qt_carbapenemase_indeterminado": 0,
                "qt_bluecarba_negativo": 0,
                "qt_imunocromatografia_negativo": 0,
                "qt_vre_negativo": qt_vre_neg,
                "qt_vre_positivo": qt_vre_pos,
                "qt_mrsa_negativo": qt_mrsa_neg,
                "qt_mrsa_positivo": qt_mrsa_pos,
                "qt_pesq_molecular_negativo": 0,
                "qt_pesq_molecular_positivo": 0,
            }
        )

    return pd.DataFrame(registros)


# ---------------------------------------------------------------------------
# Função: gerar_excel_brglass
# ---------------------------------------------------------------------------
def gerar_excel_brglass(csv_path: str, xlsx_out: str) -> None:
    """
    Converte um CSV VIGIRAM em planilha BR-GLASS.
    """
    mes, ano = _extrair_mes_ano2(csv_path)
    if mes == 0 or ano == 0:
        now = datetime.now()
        mes, ano = now.month, now.year

    df_r10, df_r20, df_r30, df_r40, df_r50 = processar_csv(csv_path)

    # 1. R60 / R70 / R80
    df_r50["id_antimicrobiano"] = df_r50["id_antimicrobiano"].astype(str)
    df_r50["id_microrganismo"] = df_r50["id_microrganismo"].astype(str)
    df_r50["rs_tsa"] = df_r50["rs_tsa"].astype(str).str.strip().str.upper()

    df_R60 = df_r50[
        (df_r50["id_antimicrobiano"] == "113")
        & (df_r50["rs_tsa"] == "R")
        & (df_r50["id_microrganismo"].isin(MIC_CARBA_VALIDOS))
    ].copy()

    df_R70 = df_r50[
        (df_r50["id_antimicrobiano"] == "176")
        & (df_r50["rs_tsa"] == "R")
        & (df_r50["id_microrganismo"].isin(MIC_VRE_VALIDOS))
    ].copy()

    df_R80 = df_r50[
        (df_r50["id_antimicrobiano"] == "131")
        & (df_r50["rs_tsa"] == "R")
        & (df_r50["id_microrganismo"] == MIC_MRSA_VALIDO)
    ].copy()

    def _prep(df, cols):
        return df[cols] if not df.empty else df_vazio(cols)

    df_R60 = _prep(
        df_R60.assign(
            nr_registro="60",
            co_metodo_carbapenemase=1,
            rs_teste_carbapenemase="1",
            co_carbapenemase=3,
        ),
        [
            "nr_registro",
            "id_estabelecimento",
            "co_atendimento",
            "co_amostra",
            "id_microrganismo",
            "co_metodo_carbapenemase",
            "rs_teste_carbapenemase",
            "co_carbapenemase",
        ],
    )

    df_R70 = _prep(
        df_R70.assign(nr_registro="70", rs_vre="1"),
        [
            "nr_registro",
            "id_estabelecimento",
            "co_atendimento",
            "co_amostra",
            "id_microrganismo",
            "rs_vre",
        ],
    )

    df_R80 = _prep(
        df_R80.assign(nr_registro="80", rs_mrsa="1"),
        [
            "nr_registro",
            "id_estabelecimento",
            "co_atendimento",
            "co_amostra",
            "id_microrganismo",
            "rs_mrsa",
        ],
    )

    # 2. Consolidado diário
    df_consolidado = gerar_consolidado_diario(
        df_r30, df_R60, df_R70, df_R80, df_r50, mes_alvo=mes, ano_alvo=ano
    )

    # 3. Abas vazias
    df_pesq = df_vazio(
        [
            "nr_registro",
            "id_estabelecimento",
            "co_atendimento",
            "co_amostra",
            "id_microrganismo",
            "rs_metodo_pesquisa_molecular",
            "rs_gene",
            "rs_outro_gene",
        ]
    )
    df_erros = df_vazio(["data"])
    df_br = df_vazio([])

    # 4. Gravação
    with pd.ExcelWriter(xlsx_out, engine="openpyxl") as wr:
        df_estabelecimento(mes, ano).to_excel(
            wr, sheet_name="Estabelecimento", index=False
        )
        df_r10.to_excel(wr, sheet_name="Paciente", index=False)
        df_r20.to_excel(wr, sheet_name="Atendimento", index=False)
        df_r30.to_excel(wr, sheet_name="Amostra", index=False)
        df_r40.to_excel(wr, sheet_name="Microrganismo", index=False)
        df_r50.to_excel(wr, sheet_name="TSA", index=False)
        df_R60.to_excel(wr, sheet_name="Carbapenemase", index=False)
        df_R70.to_excel(wr, sheet_name="VRE", index=False)
        df_R80.to_excel(wr, sheet_name="MRSA", index=False)
        df_pesq.to_excel(wr, sheet_name="Pesquisa Molecular", index=False)
        df_consolidado.to_excel(wr, sheet_name="Consolidado diário", index=False)
        df_erros.to_excel(wr, sheet_name="Erros", index=False)
        df_br.to_excel(wr, sheet_name="BRGLASS", index=False)

    formatar_planilha(xlsx_out)
    print(f"✔ Gerado: {xlsx_out}")

In [ ]:
# =========================================================
# BLOCO 11 · processar_todos_arquivos_ano (opcional)
# =========================================================
def processar_todos_arquivos_ano(ano_2d: str):
    """
    Procura em IN_DIR arquivos do tipo vigiram-???{ano_2d}.csv
    e grava os XLSX finais em OUT_DIR.
    """
    import re, os

    r = re.compile(rf"vigiram-[a-z]{{3}}{ano_2d}\.csv$", re.IGNORECASE)

    for nome_arq in os.listdir(IN_DIR):
        if r.match(nome_arq):
            caminho_csv = IN_DIR / nome_arq
            base = os.path.splitext(nome_arq)[0]  # ex: vigiram-jan25
            xlsx_out = OUT_DIR / f"Copia_de_Modelo_BR-GLASS_{base}.xlsx"
            gerar_excel_brglass(caminho_csv, xlsx_out)

    print("✔ Finalizado para todos os arquivos do ano 20" + ano_2d)

In [42]:
# =========================================================
# BLOCO 12 · EXEMPLO DE USO
# =========================================================
# 1 arquivo:
# gerar_excel_brglass("vigiram-jul24.csv", "Copia_de_Modelo_BR-GLASS_vigiram-jul24.xlsx")
# Para processar todos os arquivos do ano, descomente a linha abaixo:
processar_todos_arquivos_ano("25")

✔ Gerado: C:\Users\marcelo.petry\Documents\Vigiram\outputs\Copia_de_Modelo_BR-GLASS_vigiram-abr25.xlsx
✔ Gerado: C:\Users\marcelo.petry\Documents\Vigiram\outputs\Copia_de_Modelo_BR-GLASS_vigiram-fev25.xlsx
✔ Gerado: C:\Users\marcelo.petry\Documents\Vigiram\outputs\Copia_de_Modelo_BR-GLASS_vigiram-jan25.xlsx
✔ Gerado: C:\Users\marcelo.petry\Documents\Vigiram\outputs\Copia_de_Modelo_BR-GLASS_vigiram-jun25.xlsx
✔ Gerado: C:\Users\marcelo.petry\Documents\Vigiram\outputs\Copia_de_Modelo_BR-GLASS_vigiram-mai25.xlsx
✔ Gerado: C:\Users\marcelo.petry\Documents\Vigiram\outputs\Copia_de_Modelo_BR-GLASS_vigiram-mar25.xlsx
✔ Finalizado para todos os arquivos do ano 2025
